In [39]:
import os

In [40]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [41]:
os.chdir("../.")

In [42]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [43]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TablesConfig:
    customers: str
    products: str

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    raw_data_dir: Path
    customers_csv: Path
    products_csv: Path
    

In [45]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,    
        params_filepath: str = PARAMS_FILE_PATH,
        #schema_filepath: str = SCHEMA_FILE_PATH,
        ):
       
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)        
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([self.config.artifacts_root])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            raw_data_dir=Path(config.raw_data_dir),
            customers_csv=Path(config.customers_csv),
            products_csv=Path(config.products_csv),
            
        )
        
        return data_ingestion_config

In [47]:
import os
import logging
import sqlite3
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from BankProducts import logger

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_and_save_data(self):
        """Extract customers and products data from SQLite DB and save as CSV."""
        logging.info(f"Connecting to database: {self.config.db_file}")
        conn = sqlite3.connect(self.config.db_file)

        try:
            customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.customers_table}", conn)
            products_df = pd.read_sql_query(f"SELECT * FROM {self.config.products_table}", conn)
            logging.info(f"Successfully read tables {self.config.customers_table} and {self.config.products_table}")
        finally:
            conn.close()
            logging.info("Database connection closed.")

        # Ensure directories exist
        Path(self.config.raw_data_dir).mkdir(parents=True, exist_ok=True)

        # Save to CSV
        customers_df.to_csv(self.config.customers_csv, index=False)
        products_df.to_csv(self.config.products_csv, index=False)
        logging.info(f"Customers data saved to: {self.config.customers_csv}")
        logging.info(f"Products data saved to: {self.config.products_csv}")
        
        return customers_df, products_df

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_and_save_data()
except Exception as e:
    logger.exception(e)
    raise e


[2025-05-17 21:40:32,474: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-17 21:40:32,479: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-17 21:40:32,483: INFO: common: created directory at: artifacts]
[2025-05-17 21:40:32,487: INFO: common: created directory at: artifacts]
[2025-05-17 21:40:32,490: ERROR: 930908772: "'ConfigBox' object has no attribute 'table'"]
Traceback (most recent call last):
  File "box\box.py", line 503, in box.box.Box.__getitem__
KeyError: 'table'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\box.py", line 536, in box.box.Box.__getattr__
  File "box\box.py", line 524, in box.box.Box.__getitem__
box.exceptions.BoxKeyError: "'table'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "box\box.py", line 538, in box.box.Box.__getattr__
AttributeError: 'ConfigBox' object has no attribute 't

BoxKeyError: "'ConfigBox' object has no attribute 'table'"